Libraries and imports.

In [1]:
import pickle

import numpy as np
from PIL import Image
from tensorflow import keras

Dictionary for translating a protein into an image.

In [2]:
to_list_dict = {'A': [1,0,0,0], 'C': [0,1,0,0], 'G': [0,0,1,0], 'T':[0,0,0,1]}

The model filename defines the already trained model to be used here.
The mod1 filename defines the processed data of the genbank dataset and contains for each sequence:
- The complete original sequence that will be used to predict the final sequence with the CNN.
- The complete final sequence that is expected as result or a.k.a. y data.

In [3]:
model_filename_input = "actin"
mod1_filename_input = "actin"

Loading the model to be used

In [ ]:
model = keras.models.load_model(f"../assets/models/{model_filename_input}.h5")

Function to convert DNA sequence to image.

In [5]:
def convert_to_image(array):
  image = []
  for char in array:
    image.append(to_list_dict[char])
  
  image = np.array(image, np.uint8)

  image = Image.fromarray(image)
  
  resized = image.resize((4, 121))

  return np.array(resized)

Given a sequence x from a dataset X, first the first start of a possible intron is found and then the first possible end for it. The possible intron is then fed into the model, if it is predicted to be a real intron, the intron position is saved in the list (to be removed later) and then the process of finding possible intron combinations continues from the next possible 'GT' position. If the model predict it as not an intron, the search simply continues finding the next possible combination of 'GT...AG' until it is no longer possible.

In [6]:
def predict_sequence(seq):
  seq = seq.upper()

  intron_comb = []
  gt_pos = 0
  ag_pos = 0

  intron_comb = []
  gt_pos = seq.find("GT")
  while(gt_pos != -1):
    ag_pos = seq.find("AG", gt_pos+2)
    prediction = 1
    while(ag_pos != -1):
      current_seq = seq[gt_pos:ag_pos+2]

      image_to_predict = convert_to_image(current_seq)

      prediction = model.predict(np.array([image_to_predict]), verbose=0)
      prediction = round(prediction[0][0])

      if prediction == 0:
        intron_comb.append(current_seq)
        gt_pos = seq.find("GT", ag_pos+1)
        ag_pos = -1
      else:
        ag_pos = seq.find("AG", ag_pos+1)

    if prediction != 0:
      gt_pos = seq.find("GT", gt_pos+1)

  for intron in intron_comb:
    seq = seq.replace(intron, "")

  return seq

Gathering the module 1 data and using it to compare the results.

In [7]:
# Credits to "slowkow" from "https://gist.github.com/slowkow/06c6dba9180d013dfd82bec217d22eb5"

def nw(x, y, match = 1, mismatch = 1, gap = 1):
    nx = len(x)
    ny = len(y)
    # Optimal score at each possible pair of characters.
    F = np.zeros((nx + 1, ny + 1))
    F[:,0] = np.linspace(0, -nx * gap, nx + 1)
    F[0,:] = np.linspace(0, -ny * gap, ny + 1)
    # Pointers to trace through an optimal aligment.
    P = np.zeros((nx + 1, ny + 1))
    P[:,0] = 3
    P[0,:] = 4
    # Temporary scores.
    t = np.zeros(3)
    for i in range(nx):
        for j in range(ny):
            if x[i] == y[j]:
                t[0] = F[i,j] + match
            else:
                t[0] = F[i,j] - mismatch
            t[1] = F[i,j+1] - gap
            t[2] = F[i+1,j] - gap
            tmax = np.max(t)
            F[i+1,j+1] = tmax
            if t[0] == tmax:
                P[i+1,j+1] += 2
            if t[1] == tmax:
                P[i+1,j+1] += 3
            if t[2] == tmax:
                P[i+1,j+1] += 4
    # Trace through an optimal alignment.
    i = nx
    j = ny
    rx = []
    ry = []
    while i > 0 or j > 0:
        if P[i,j] in [2, 5, 6, 9]:
            rx.append(x[i-1])
            ry.append(y[j-1])
            i -= 1
            j -= 1
        elif P[i,j] in [3, 5, 7, 9]:
            rx.append(x[i-1])
            ry.append('-')
            i -= 1
        elif P[i,j] in [4, 6, 7, 9]:
            rx.append('-')
            ry.append(y[j-1])
            j -= 1
    # Reverse the strings.
    rx = ''.join(rx)[::-1]
    ry = ''.join(ry)[::-1]
    return '\n'.join([rx, ry])

In [8]:
file = open("../assets/mod1/"+mod1_filename_input+".mod1", "rb")
processed_database = pickle.load(file)['test']

In [ ]:
normalization_result = []
database_len = len(processed_database)
progress = 1

for sequence in processed_database:
  predicted_result = predict_sequence(sequence['complete_sequence'])
  comparation_result = nw(predicted_result, sequence['response_sequence'])
  a, b = comparation_result.split('\n')

  max_sequence_size = max(len(a), len(b))
  min_sequence_size = min(len(a), len(b))

  hits = 0
  for i in range(min_sequence_size):
    if a[i] == b[i]:
      hits += 1
  normalization_result.append(hits/max_sequence_size)
  
  normalization_result_array = np.array(normalization_result)
  mean = normalization_result_array.mean()

  print(f"Progress: {progress}/{database_len} - Mean: {mean}")
  progress += 1


In [ ]:
normalization_result_array = np.array(normalization_result)

In [ ]:
mean = normalization_result_array.mean()

In [ ]:
def distribution(array):
  limits = np.arange(0.0, 1.1, 0.1)
  dict_for_dict = {}
  
  for i in range(len(limits) - 1):
    floor = limits[i]
    ceil = limits[i + 1]
    count = np.sum((array > floor) & (array <= ceil))
    interval = f"({floor:.1f},{ceil:.1f}]"
    dict_for_dict[interval] = int(count)
    
  return dict_for_dict

In [ ]:
distribution_dict = distribution(normalization_result_array)

In [ ]:
distribution_dict

In [ ]:
file = open("../assets/eval/"+model_filename_input+".eval","wb")
pickle.dump({'normalization_result': normalization_result, 'mean': mean, 'distribution': distribution_dict}, file)

If already exists evaluation file

In [ ]:
file = open("../assets/eval/"+model_filename_input+".eval","rb")
data = pickle.load(file)

In [ ]:
data['mean']